# DB 업로드(.csv 파일 만들기)

In [ ]:
import pandas as pd
import glob
import numpy as np
import os  # 로컬 경로 제어를 위해 추가

# [환경설정] 데이터 파일이 있는 폴더 경로로 이동
# 예: r'C:\Users\Documents\Project\Data' (경로 앞에 r을 붙이면 역슬래시 에러 방지)
data_path = r'C:\Users\Julia\Downloads\raw_data'
os.chdir(data_path)

# 1. 선생님이 확정한 업종 및 컬럼 리스트
target_sectors = [
    '섬유제품', '완구', '운동/경기_용품', '화장품', '문구', '서적', 
    '시계및귀금속', '안경', '일반의류', '편의점', '노래방', '미용실', 
    '당구장', '커피-음료', '호프-간이주점', '분식전문점', '치킨전문점', 
    '패스트푸드점', '제과점', '양식음식점', '중식음식점', '한식음식점'
]

cols_to_keep = [
    '기준_년분기_코드', '행정동_코드', '행정동_코드_명', '서비스_업종_코드_명', 
    '당월_매출_금액', '주중_매출_금액', '주말_매출_금액', 
    '연령대_10_매출_금액', '연령대_20_매출_금액', '연령대_30_매출_금액'
]

# 2. 연도별 매출 데이터 통합 (추정매출만 연도별)
print("진행 중: 연도별 추정매출 통합...")
sales_files = sorted(glob.glob('매출_*.csv')) # 파일명 규칙 확인 필요
sales_list = []

for f in sales_files:
    # 불러올 때부터 필요한 컬럼만 추출하여 메모리 확보
    df = pd.read_csv(f, usecols=cols_to_keep, encoding='cp949')
    # 업종 필터링
    df = df[df['서비스_업종_코드_명'].isin(target_sectors)]
    sales_list.append(df)

df_sales = pd.concat(sales_list, ignore_index=True)

# 3. 단일 파일 데이터 로드 (유동인구, 상주인구, 상권지표, 집객시설)
# 각 파일의 컬럼명은 데이터 광장의 표준명칭을 기준으로 했습니다.
print("진행 중: 기타 테이블 병합...")

# [주의] 파일명은 로컬에 저장된 이름과 똑같아야 합니다!
df_pop = pd.read_csv(r'C:\Users\Julia\Downloads\raw_data\유동인구.csv', encoding='cp949')
df_resident = pd.read_csv(r'C:\Users\Julia\Downloads\raw_data\상주인구.csv', encoding='cp949')
df_change = pd.read_csv(r'C:\Users\Julia\Downloads\raw_data\상권변화지표.csv', encoding='cp949')
df_facility = pd.read_csv(r'C:\Users\Julia\Downloads\raw_data\집객시설.csv', encoding='cp949')

# 길단위인구: MZ 유동인구 핵심
df_pop = df_pop[['기준_년분기_코드', '행정동_코드', '총_유동인구_수', '연령대_20_유동인구_수', '연령대_30_유동인구_수']]
df_pop['MZ_유동인구'] = df_pop['연령대_20_유동인구_수'] + df_pop['연령대_30_유동인구_수']

# 상주인구: 베드타운 지수용 
df_resident = df_resident[['기준_년분기_코드', '행정동_코드', '총_상주인구_수', '총_가구_수']]

# 집객시설: 인프라 위주
df_facility = df_facility[['기준_년분기_코드', '행정동_코드', '집객시설_수', '지하철_역_수']]

# 상권변화지표: 역동성 스코어링 포함
df_change = df_change[['기준_년분기_코드', '행정동_코드', '상권_변화_지표_명', '운영_영업_개월_평균']]
# 아까 정한 1~4점 매핑 적용
mapping = {'다이나믹': 4, '상권확장': 3, '정체': 2, '상권축소': 1}
df_change['상권지표_점수'] = df_change['상권_변화_지표_명'].map(mapping).fillna(0)

# 4. 최종 Merge (기준_년분기_코드와 행정동_코드를 키로 활용)
final_df = df_sales.merge(df_pop, on=['기준_년분기_코드', '행정동_코드'], how='left')
final_df = final_df.merge(df_resident, on=['기준_년분기_코드', '행정동_코드'], how='left')
final_df = final_df.merge(df_change, on=['기준_년분기_코드', '행정동_코드'], how='left')
final_df = final_df.merge(df_facility, on=['기준_년분기_코드', '행정동_코드'], how='left')

# 5. 결과 저장
final_df.fillna(0, inplace=True)
final_df.to_csv('final_alpha_data.csv', index=False, encoding='utf-8-sig')
print("축하합니다! 분석용 최종 데이터 셋 생성이 완료되었습니다.")

In [ ]:
# 1. 행 개수 비교
original_rows = len(df_sales)
final_rows = len(final_df)

print(f"--- [1. 데이터 손실 검증] ---")
print(f"매출 데이터 원본 행 수: {original_rows}")
print(f"최종 병합 데이터 행 수: {final_rows}")

if original_rows == final_rows:
    print("✅ 성공: 데이터 누락이나 중복 생성 없이 완벽하게 병합되었습니다.")
else:
    print("⚠️ 주의: 행 개수가 다릅니다. 중복 데이터(Duplication)가 있는지 확인이 필요합니다.")

In [ ]:
# 1. 데이터 상단 및 구조 확인
print("--- [1. 데이터 기본 구조] ---")
print(final_df.info()) 

# 2. 요약 통계량 확인 (매출, 인구, 지수 등이 상식적인 범위인지)
print("\n--- [2. 주요 지표 요약 통계] ---")
# 분석에 핵심적인 컬럼들만 골라서 봅니다.
key_cols = ['당월_매출_금액', '총_유동인구_수', '총_상주인구_수', 'MZ_유동인구', '상권지표_점수']
# 존재하는 컬럼만 필터링해서 확인
existing_cols = [c for c in key_cols if c in final_df.columns]
print(final_df[existing_cols].describe())

# 3. 데이터 중복 여부 확인
# 동일 분기에 동일 행정동, 동일 업종이 두 번 들어가면 안 됩니다.
duplicate_count = final_df.duplicated(subset=['기준_년분기_코드', '행정동_코드', '서비스_업종_코드_명']).sum()
print(f"\n--- [3. 중복 데이터 체크] ---")
print(f"중복된 행(Row) 개수: {duplicate_count}개")

# 4. '가양동' vs '성수동' 극명한 차이 확인 (Spot Check)
print(f"\n--- [4. 베드타운 vs 핫플레이스 비교 검증] ---")
comparison = final_df[final_df['행정동_코드_명'].isin(['가양1동', '성수2가1동'])].groupby('행정동_코드_명')[existing_cols].mean()
print(comparison)

In [ ]:
# 1. 효율 지표 생성
# 배후 인구(상주인구) 대비 얼마나 외부에서 많이 오나? (상권 효율성)
final_df['상권_유입_강도'] = final_df['총_유동인구_수'] / (final_df['총_상주인구_수'] + 1)

# 2. MZ 타겟팅 지표
# 전체 유동인구 중 MZ(2030)가 차지하는 비율
final_df['MZ_유입_비중'] = (final_df['연령대_20_유동인구_수'] + final_df['연령대_30_유동인구_수']) / (final_df['총_유동인구_수'] + 1)

# 3. 데이터 포인트 최신화 (2025년 1분기 기준)
analysis_2025 = final_df[final_df['기준_년분기_코드'] == 20251].copy()

In [ ]:
!uv pip install matplotlib seaborn

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.preprocessing import MinMaxScaler

# 1. 데이터 준비 및 전처리
df_analysis = final_df.copy()
df_analysis['행정동_코드_명'] = df_analysis['행정동_코드_명'].str.replace(r'[^가-힣0-9]', '', regex=True)

# 2. 행정동별/분기별 그룹화
df_grouped = df_analysis.groupby(['기준_년분기_코드', '행정동_코드_명']).agg({
    '당월_매출_금액': 'sum',
    'MZ_유동인구': 'sum',
    '총_유동인구_수': 'sum',
    '연령대_20_매출_금액': 'sum',
    '연령대_30_매출_금액': 'sum',
    '집객시설_수': 'max', 
    '지하철_역_수': 'max',
    '운영_영업_개월_평균': 'mean'
}).reset_index().sort_values(['행정동_코드_명', '기준_년분기_코드'])

# 3. 타겟 변수: MZ 유동인구 증가율
df_grouped['MZ_유동_증가율'] = df_grouped.groupby('행정동_코드_명')['MZ_유동인구'].pct_change()
df_grouped = df_grouped.replace([np.inf, -np.inf], np.nan).dropna(subset=['MZ_유동_증가율'])

# 4. 독립변수 재구성 (MZ 관련 모든 변수 제외)
# 상권_에너지_지수 계산 시 '당월_매출_금액'은 전체 매출이므로 유지
df_grouped['상권_에너지_지수'] = df_grouped['당월_매출_금액'] * df_grouped['집객시설_수']

# 5. 정규화 및 최종 변수 선정
scaler = MinMaxScaler()
# 순수 상권 환경 변수들로만 구성
cols = ['집객시설_수', '지하철_역_수', '운영_영업_개월_평균', '상권_에너지_지수']
df_grouped[cols] = scaler.fit_transform(df_grouped[cols])

# 6. OLS 회귀분석 실행
X = df_grouped[cols]
X = sm.add_constant(X)
Y = df_grouped['MZ_유동_증가율']

model = sm.OLS(Y, X).fit()
weights = model.params

# 7. 최종 지수 산출 (통계 가중치 반영)
df_grouped['최종_지수'] = sum(df_grouped[col] * weights[col] for col in cols)

# 8. 최종 랭킹 및 101위 리스트업
# 전체 기간에 대한 행정동별 평균 점수로 랭킹 산정
final_ranking = df_grouped.groupby('행정동_코드_명')['최종_지수'].mean().sort_values(ascending=False).reset_index()
final_ranking.insert(0, 'Rank', range(1, len(final_ranking) + 1))

# CSV 저장 (이제 이 리스트를 들고 네이버 트렌드로 가서 검증하시면 됩니다!)
final_ranking.head(50).to_csv('NSI_9.0_Candidate_List.csv', index=False, encoding='utf-8-sig')

# 조원들에게 보여줄 통계 근거 출력
print(model.summary())